# Linear Regression: Predicting Song Popularity from Audio Features

**Goal:** Use linear regression to predict Spotify song popularity (0-100) from audio features like danceability, energy, and loudness.

**Personal Note:** As someone with 11 years of ballroom dancing experience, I love exploring how audio features relate to what makes music popular!

In [ ]:
STUDENT_NUMBER = 15854493

## 1. Setup and Load Data

In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import statsmodels.api as sm

In [ ]:
# load data
custom_data_path = "sem3_topic3_linreg_summative_data.csv"
custom_df = pd.read_csv(custom_data_path)

df = pd.read_csv("sem3_topic3_linreg_summative_data.csv")
print(f"Dataset: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"\nMissing values: {df.isnull().sum().sum()}")
df.head(3)

## 2. Data Cleaning

Remove rows with missing values and drop identifier columns that aren't useful for prediction.

In [ ]:
# drop rows with any missing values (only 3 rows)
df = df.dropna()

# drop identifier columns that won't help prediction
df = df.drop(columns=["Unnamed: 0", "track_id", "track_name", "album_name", "artists"])

print(f"Clean dataset: {df.shape[0]} rows, {df.shape[1]} columns")

## 3. Check Multicollinearity (Correlation Matrix)

**Why?** Highly correlated predictors (multicollinearity) can make linear regression coefficients unstable and hard to interpret. We check for pairs with |correlation| > 0.85.

In [ ]:
# select only numeric columns for correlation
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
corr_matrix = df[numeric_cols].corr()

# plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# check for highly correlated pairs (threshold = 0.85)
print("Checking for multicollinearity (|correlation| > 0.85):")
high_corr_found = False

for i in range(len(numeric_cols)):
    for j in range(i + 1, len(numeric_cols)):
        corr_value = corr_matrix.iloc[i, j]
        if abs(corr_value) > 0.85:
            print(f"  {numeric_cols[i]} & {numeric_cols[j]}: {corr_value:.3f}")
            high_corr_found = True

if not high_corr_found:
    print("  No highly correlated pairs found - no features need removal.")

**Interpretation:** No feature pairs exceed the 0.85 threshold, so we keep all features. However, we observe:
- Energy & loudness have moderate positive correlation (~0.77) - louder songs tend to be more energetic
- Energy & acousticness have negative correlation (~-0.73) - energetic songs tend not to be acoustic
- Popularity has weak correlations with all features - suggesting non-linear relationships may exist

## 4. Train/Validation/Test Split

**Why 3 splits?**
- **Training (70%):** Model learns patterns
- **Validation (15%):** Compare models, tune hyperparameters
- **Test (15%):** Final unbiased evaluation (only used once at the end)

In [ ]:
# define features (X) and target (y)
X = df.drop(columns=["popularity"])
y = df["popularity"]

# first split: 70% train, 30% temp
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)

# second split: 50/50 of temp = 15% validation, 15% test
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)

print(f"Training set:   {X_train.shape[0]} samples")
print(f"Validation set: {X_valid.shape[0]} samples")
print(f"Test set:       {X_test.shape[0]} samples")

## 5. Preprocessing Pipeline

The `track_genre` column is categorical (e.g., "pop", "rock"). We use One-Hot Encoding to convert it to numeric columns.

In [ ]:
# identify categorical vs numeric columns
cat_cols = ["track_genre"]
num_cols = [col for col in X.columns if col not in cat_cols]

# create preprocessor: one-hot encode genre, pass numeric through unchanged
preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ("num", "passthrough", num_cols)
])

## 6. Model 1: Linear Regression

In [ ]:
# create pipeline: preprocessing + linear regression
linreg_pipe = Pipeline([
    ("prep", preprocessor),
    ("model", LinearRegression())
])

# fit on training data
linreg_pipe.fit(X_train, y_train)

# predict on validation set
pred_lr = linreg_pipe.predict(X_valid)

# calculate metrics
mae_lr = mean_absolute_error(y_valid, pred_lr)
rmse_lr = np.sqrt(mean_squared_error(y_valid, pred_lr))
r2_lr = r2_score(y_valid, pred_lr)

print("Linear Regression (Validation Set)")
print(f"  MAE:  {mae_lr:.2f}")
print(f"  RMSE: {rmse_lr:.2f}")
print(f"  R²:   {r2_lr:.4f}")

### 6.1 Coefficient Analysis (Statistical Significance)

We use statsmodels to get p-values and check which predictors have statistically significant relationships with popularity.

In [ ]:
# transform training data for statsmodels
X_train_transformed = preprocessor.fit_transform(X_train)
feature_names = preprocessor.get_feature_names_out()

# add constant (intercept) for statsmodels
X_train_sm = sm.add_constant(X_train_transformed)

# fit OLS model
ols_model = sm.OLS(y_train, X_train_sm).fit()

# show summary for numeric features only (genres are too many)
# extract coefficients for numeric features
coef_df = pd.DataFrame({
    'Feature': ['const'] + list(feature_names),
    'Coefficient': ols_model.params,
    'p-value': ols_model.pvalues
})

# filter to show numeric features
numeric_features = coef_df[coef_df['Feature'].str.startswith(('const', 'num__'))]
print("Coefficient Significance (numeric features):")
print(numeric_features.to_string(index=False))
print("\nNote: p-value < 0.05 indicates statistical significance")

**Interpretation:** 
- **Loudness** has the largest positive coefficient - louder songs tend to be more popular
- **Danceability** also has a positive effect on popularity
- Most coefficients are small and some may not be statistically significant (p > 0.05)
- The low R² (close to 0) tells us linear relationships explain very little of the variance in popularity

### 6.2 Residual Analysis

**Why?** Residuals (actual - predicted) should be randomly distributed around zero. Patterns in residuals suggest the model is missing something.

In [ ]:
# calculate residuals
residuals_lr = y_valid - pred_lr

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# plot 1: residuals vs predicted values
axes[0].scatter(pred_lr, residuals_lr, alpha=0.3, s=10)
axes[0].axhline(y=0, color='red', linestyle='--')
axes[0].set_xlabel('Predicted Popularity')
axes[0].set_ylabel('Residual')
axes[0].set_title('Residuals vs Predicted Values')

# plot 2: histogram of residuals
axes[1].hist(residuals_lr, bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(x=0, color='red', linestyle='--')
axes[1].set_xlabel('Residual')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Residuals')

plt.tight_layout()
plt.show()

print(f"Mean of residuals: {residuals_lr.mean():.4f} (should be ~0)")
print(f"Std of residuals:  {residuals_lr.std():.2f}")

**Interpretation:** 
- The residuals are roughly centred around zero (mean ≈ 0), which is good
- However, the scatter plot shows a pattern: residuals are NOT randomly distributed
- This suggests a linear model is not capturing the true relationship - there may be non-linear patterns
- This motivates trying a more flexible model like Random Forest

## 7. Model 2: Random Forest

**Why Random Forest?** The correlation matrix and residual analysis suggest non-linear relationships. Random Forest can capture these without assuming linearity.

In [ ]:
# create pipeline: preprocessing + random forest
rf_pipe = Pipeline([
    ("prep", preprocessor),
    ("model", RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1))
])

# fit on training data
rf_pipe.fit(X_train, y_train)

# predict on validation set
pred_rf = rf_pipe.predict(X_valid)

# calculate metrics
mae_rf = mean_absolute_error(y_valid, pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_valid, pred_rf))
r2_rf = r2_score(y_valid, pred_rf)

print("Random Forest (Validation Set)")
print(f"  MAE:  {mae_rf:.2f}")
print(f"  RMSE: {rmse_rf:.2f}")
print(f"  R²:   {r2_rf:.4f}")

## 8. Model Comparison

**Metrics explained:**
- **MAE (Mean Absolute Error):** Average prediction error in popularity points. Lower = better.
- **RMSE (Root Mean Squared Error):** Similar to MAE but penalises large errors more. Lower = better.
- **R² (R-squared):** Proportion of variance explained (0-1). Higher = better. R²=0 means no better than predicting the mean.

In [ ]:
# create comparison table
results = pd.DataFrame({
    "Model": ["Linear Regression", "Random Forest"],
    "MAE": [mae_lr, mae_rf],
    "RMSE": [rmse_lr, rmse_rf],
    "R²": [r2_lr, r2_rf]
})

print("Model Comparison (Validation Set):")
print(results.to_string(index=False))

**Interpretation:**

| Metric | Linear Regression | Random Forest | Winner |
|--------|-------------------|---------------|--------|
| MAE    | ~17.2 | ~13.9 | Random Forest (lower error) |
| RMSE   | ~20.4 | ~17.9 | Random Forest (lower error) |
| R²     | ~0.006 | ~0.24 | Random Forest (explains 24% vs 0.6% of variance) |

**Conclusion:** Random Forest significantly outperforms Linear Regression. The near-zero R² for Linear Regression confirms that popularity is NOT linearly related to audio features. Random Forest captures non-linear patterns and achieves ~24% explained variance - modest but meaningful for predicting human preferences.

**Selected model:** Random Forest

## 9. Final Evaluation on Test Set

In [ ]:
# predict on the held-out test set (first time we use it!)
test_pred = rf_pipe.predict(X_test)

# calculate final metrics
mae_test = mean_absolute_error(y_test, test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, test_pred))
r2_test = r2_score(y_test, test_pred)

print("Final Model Evaluation (Test Set):")
print(f"  MAE:  {mae_test:.2f}")
print(f"  RMSE: {rmse_test:.2f}")
print(f"  R²:   {r2_test:.4f}")

**Interpretation:**
- **MAE (~14):** On average, predictions are about 14 points off on a 0-100 scale
- **RMSE (~18):** Slightly higher due to some larger errors
- **R² (~0.23):** Model explains about 23% of variance - consistent with validation results

The test set performance is very close to validation performance, indicating the model generalises well and we didn't overfit during model selection.

## 10. Generalisation Error with Confidence Interval (Bootstrap)

**Why bootstrap?** A single test set gives us one estimate of error. Bootstrap resampling gives us a distribution of errors, allowing us to calculate a confidence interval.

In [ ]:
# bootstrap to get confidence interval for RMSE
np.random.seed(42)
n_bootstrap = 1000
bootstrap_rmse = []

for _ in range(n_bootstrap):
    # sample with replacement from test set
    indices = np.random.choice(len(y_test), size=len(y_test), replace=True)
    y_sample = y_test.iloc[indices]
    pred_sample = test_pred[indices]
    
    # calculate RMSE for this sample
    rmse_sample = np.sqrt(mean_squared_error(y_sample, pred_sample))
    bootstrap_rmse.append(rmse_sample)

# calculate 95% confidence interval
ci_lower = np.percentile(bootstrap_rmse, 2.5)
ci_upper = np.percentile(bootstrap_rmse, 97.5)
mean_rmse = np.mean(bootstrap_rmse)

print(f"Bootstrap Results ({n_bootstrap} iterations):")
print(f"  Mean RMSE: {mean_rmse:.2f}")
print(f"  95% Confidence Interval: [{ci_lower:.2f}, {ci_upper:.2f}]")

In [ ]:
# visualise the bootstrap distribution
plt.figure(figsize=(8, 4))
plt.hist(bootstrap_rmse, bins=40, edgecolor='black', alpha=0.7)
plt.axvline(mean_rmse, color='red', linestyle='--', label=f'Mean: {mean_rmse:.2f}')
plt.axvline(ci_lower, color='orange', linestyle=':', label=f'95% CI: [{ci_lower:.2f}, {ci_upper:.2f}]')
plt.axvline(ci_upper, color='orange', linestyle=':')
plt.xlabel('RMSE')
plt.ylabel('Frequency')
plt.title('Bootstrap Distribution of Generalisation Error')
plt.legend()
plt.tight_layout()
plt.show()

**Interpretation:**
- The mean bootstrapped RMSE (~18.0) matches our test set RMSE, confirming our estimate is reliable
- The 95% confidence interval is narrow ([~17.8, ~18.2]), meaning the error is stable across different samples
- We can be 95% confident that the true generalisation error (RMSE) falls within this interval
- The model shows **reliable generalisation** - performance on unseen data is predictable

## Summary

**Key Findings:**
1. **No multicollinearity issues** - no feature pairs exceeded the 0.85 correlation threshold
2. **Linear regression performed poorly** (R² ≈ 0.006) - popularity is not linearly related to audio features
3. **Random Forest significantly outperformed** linear regression (R² ≈ 0.24)
4. **Generalisation error is stable** - RMSE confidence interval: [17.8, 18.2]

**Why only 24% explained variance?** Song popularity depends heavily on factors NOT in our data:
- Artist fame and marketing
- Playlist placements
- Social media trends
- Release timing

Audio features alone cannot fully predict these human and social factors.